In [1]:
import numpy as np
import math as m
import pandas as pd
import SatelliteClass as sc
import TerminalClass as tc
import ChannelClass as cc
import VectorClass as vc #new

from skyfield.api import load, wgs84 #new

def LinkBudget(destination, origin, bandwidth, elevation):
    k_boltz = -228.6
    testCC = cc.Channel(destination, origin, bandwidth, elevation)
    SNR = testCC.EIRP - testCC.BandWidth - testCC.get_FSPL() - testCC.get_AtmosLoss() + testCC.G_T - k_boltz
    #print(testCC.BandWidth)
    return SNR

#testSC = sc.Satellite(40000, 0, 0, 0, 0, -1)
#testTC = tc.Terminal(1800, 0, 0, 0, 0, 1)
#ans = LinkBudget(testTC, testSC, 36*10**(6))

#print(ans)


####Testing Integration
stations_url = 'OneWebDatabase.txt'
satellites = load.tle_file(stations_url)
print('Loaded', len(satellites), 'satellites')

ts = load.timescale()
t0 = ts.utc(2023, 7, 24, 0, 0, 0)
t1 = ts.utc(2023, 7, 25)

cities={'London':[51.5072, -0.1276],'Birmingham':[52.4862, -1.8904]}
#cities={'London':[51.5072, -0.1276],'Birmingham':[52.4862, -1.8904], 'Belfast': [54.5973, -5.9301], 'Cardiff': [51.4837, -3.1681], 'Edinburgh': [55.9533, -3.1883], 'Warrington': [53.3900, -2.5970] 'Baldock': [51.9895, -0.1891], 'Manchester': [53.4808, -2.2426]}

Locations = {}
for city,latlon in zip(cities.keys(),cities.values()):
    Locations[city]=wgs84.latlon(*latlon)

print(Locations)

Visibility = {}
for city, place in zip(Locations.keys(), Locations.values()):
    Visibility[city] = [vc.Vector(*(place.at(t0).position.km), 'cartesian'), 
                        {sat.name: [vc.Vector(*((sat - place).at(t0).position.km), 'cartesian'), 
                                    (sat - place).at(t0).altaz()[0].degrees, vc.Vector(*(sat.at(t0).position.km), 'cartesian')] for sat in satellites if (sat - place).at(t0).altaz()[0].degrees > 25}
                        ]

#print(Visibility)
#print(Visibility['London'][0].cartesian())
something = {}
for sat in satellites: #This is defintely the correct type of data structure I want to make!
    #something[sat.name] = {"Skyfield": sat, "SatClass": sc.Satellite(*(sat.at(t0).position.km), 'cartesian')}
    something[sat.name] = {"Skyfield": sat, "SatClass": sc.Satellite(sat.name, downlink=11.7, uplink=14.25, eirp=-13.4, g_t=-1)}

## This part would be in a time loop
#poop = [[i // 60, i % 60] for i in range(0, 24*60)]
#print(poop[1])
#t = ts.utc(2023, 7, 24, range(0, 24, for i ))
t = ts.utc(2023, 7, 24, 0, range(0, 11)) #24*60



Loaded 636 satellites
{'London': <GeographicPosition WGS84 latitude +51.5072 N longitude -0.1276 E elevation 0.0 m>, 'Birmingham': <GeographicPosition WGS84 latitude +52.4862 N longitude -1.8904 E elevation 0.0 m>}


In [2]:
from tqdm import tqdm

column_list = ["Time_Index", "Time_in_UTC", "Ground_Station", "GS_Vector", "Serving_Satellite", "SS_Vector", "GS-SS_vector", "Elevation_Angle", "Distance_from_GS_(km)", "SNR_DL", "SNR_UL", "Visible_Sats"]
df = pd.DataFrame(columns=column_list)

for index_ti,ti in tqdm(enumerate(t)):
    if index_ti not in [0, 10]:
        continue
    
    #print(satellites[1].at(ti).position.km)
    for sat_name, classes in zip(something.keys(), something.values()):
        #something[sat_name]["SatClass"].set_Position(something[sat_name]["Skyfield"].at(ti).position.km)
        lat, long = wgs84.latlon_of(classes["Skyfield"].at(ti))
        elev = wgs84.height_of(classes["Skyfield"].at(ti))
        geo = [long.degrees, lat.degrees, elev.km]
        #print(geo)
        something[sat_name]["SatClass"].set_Position(classes["Skyfield"].at(ti).position.km, geo)

    Visible = {}
    for city, place in zip(Locations.keys(), Locations.values()):
        Visible[city] = {sat.name: [something[sat.name]["SatClass"].get_Position(), (sat-place).at(ti).altaz()[-1].km,(sat-place).at(ti).altaz()[0].degrees] for sat in satellites if (sat - place).at(ti).altaz()[0].degrees > 25}

    #print(Visible)
    max_SNRs = {}

    for city, options in zip(Visible.keys(), Visible.values()):
        testTC = tc.Terminal(city, cities[city])
        testTC.set_Position(Locations[city].at(ti).position.km)
        SNRs = {}
        for sat_name in options.keys():
            #print(options[sat_name][-1])
            SNRs[sat_name] = LinkBudget(testTC, something[sat_name]["SatClass"], 4e3, options[sat_name][-1])
            #break
        #print(SNRs)
        max_SNRs[city] = {max(SNRs, key=SNRs.get): max(SNRs.values())}
        #max_SNRs[city] = {'Downlink' : {max(SNRs, key=SNRs.get): max(SNRs.values())}}
        #max_SNRs[city]['Uplink'] = 

    for city in cities.keys():
        GS_Vect = vc.Vector(*(Locations[city].at(ti).position.km), 'cartesian')
        Ser_Sat = [dummy for dummy in max_SNRs[city].keys()]
        Ser_Sat = Ser_Sat[0]
        SS_Vect = something[Ser_Sat]["SatClass"].get_Position()
        El_angle = Visible[city][Ser_Sat][-1]
        Dist_SS = Visible[city][Ser_Sat][1]
        SS_Snr = [dummy for dummy in max_SNRs[city].values()]
        SS_Snr = SS_Snr[0]
        testTC = tc.Terminal(city, cities[city])
        testTC.set_Position(Locations[city].at(ti).position.km)
        Snr_UL = LinkBudget(something[Ser_Sat]["SatClass"], testTC, 125e6, El_angle)
        #print(testTC.Properties(El_angle))

        df.loc[df.shape[0]+1] = [index_ti, ti, city, GS_Vect, Ser_Sat, SS_Vect, GS_Vect - SS_Vect, El_angle, Dist_SS, SS_Snr, Snr_UL, {sat_name: {"Earth": something[sat_name]["SatClass"].get_Position() ,"GroundSat": GS_Vect - something[sat_name]["SatClass"].get_Position()} for sat_name in Visible[city].keys()} ]
        #df
    
#print(df)
#print(something)
#print(something["ONEWEB-0703"]["SatClass"])


11it [00:33,  3.08s/it]


In [3]:

df.to_clipboard(excel=True)
df


,Time_Index,Time_in_UTC,Ground_Station,GS_Vector,Serving_Satellite,SS_Vector,GS-SS_vector,Elevation_Angle,Distance_from_GS_(km),SNR_DL,SNR_UL,Visible_Sats
1,0,<Time tt=2460149.500800741>,London,"(6365.07943731829, 301.174428041933, 38.746192...",ONEWEB-0364,"(7554.619742764798, 302.8746498265176, 42.4069...","(1276.365890671048, 128.9376175623703, 118.745...",66.572035,1276.365891,9.494223,6.557844,"{'ONEWEB-0184': {'Earth': (7577.4266183565915,..."
2,0,<Time tt=2460149.500800741>,Birmingham,"(6364.724188592995, 299.4192099187315, 37.7621...",ONEWEB-0364,"(7554.619742764798, 302.8746498265176, 42.4069...","(1343.0871053910305, 133.9134925635856, 114.00...",59.517147,1343.087105,8.689510,5.588881,"{'ONEWEB-0184': {'Earth': (7577.4266183565915,..."
3,10,<Time tt=2460149.5077451854>,London,"(6365.07943731829, 303.6775402726618, 38.75104...",ONEWEB-0368,"(7554.824618060235, 302.34220135575754, 36.214...","(1232.6360145111812, 111.4034356617881, 156.61...",73.631193,1232.636015,9.943374,7.038232,"{'ONEWEB-0204': {'Earth': (7576.698282476043, ..."
4,10,<Time tt=2460149.5077451854>,Birmingham,"(6364.724188592993, 301.9223819894903, 37.7670...",ONEWEB-0368,"(7554.824618060235, 302.34220135575754, 36.214...","(1205.231476945039, 125.23378812121153, 151.98...",80.275045,1205.231477,10.097670,7.109956,"{'ONEWEB-0204': {'Earth': (7576.698282476043, ..."


In [8]:
import numpy as np
import pandas as pd
import scipy.constants as spc
from pathlib import Path

## Test Dataframe
#df=pd.DataFrame([[1,2,1],[2,7,6],[3,10,8],[4,15,13]],columns=['index','SNR_DL','SNR_UL'])
#df
## Actual Bandwidth
DL_bandwidth=250e6
#DL_bandwidth=4e6 #?
UL_bandwidth=125e6
mcs=pd.read_csv(Path('DVBS2-table-meta.csv'))

sE_list_DL=[]
modulation_list_DL=[]
sE_list_UL=[]
modulation_list_UL=[]
for i in df.index:
    # DL
    sE_list_DL.append(mcs[mcs.SNR<df.loc[i,'SNR_DL']].SE.max())
    modulation_list_DL.append(mcs.loc[mcs[mcs.SNR<df.loc[i,'SNR_DL']].SE.argmax(),'code'])
    #UL
    sE_list_UL.append(mcs[mcs.SNR<df.loc[i,'SNR_UL']].SE.max())
    modulation_list_UL.append(mcs.loc[mcs[mcs.SNR<df.loc[i,'SNR_UL']].SE.argmax(),'code'])

df['DL_capacity']=np.array(sE_list_DL)*DL_bandwidth/1e6
df['DL_Mode']=modulation_list_DL
df['UL_capacity']=np.array(sE_list_UL)*UL_bandwidth/1e6
df['UL_Mode']=modulation_list_UL
df

,Time_Index,Time_in_UTC,Ground_Station,GS_Vector,Serving_Satellite,SS_Vector,GS-SS_vector,Elevation_Angle,Distance_from_GS_(km),SNR_DL,SNR_UL,Visible_Sats,DL_capacity,DL_Mode,UL_capacity,UL_Mode
1,0,<Time tt=2460149.500800741>,London,"(6365.07943731829, 301.174428041933, 38.746192...",ONEWEB-0364,"(7554.619742764798, 302.8746498265176, 42.4069...","(1276.365890671048, 128.9376175623703, 118.745...",66.572035,1276.365891,9.494223,6.557844,"{'ONEWEB-0184': {'Earth': (7577.4266183565915,...",659.30025,8PSK,223.576500,QPSK
2,0,<Time tt=2460149.500800741>,Birmingham,"(6364.724188592995, 299.4192099187315, 37.7621...",ONEWEB-0364,"(7554.619742764798, 302.8746498265176, 42.4069...","(1343.0871053910305, 133.9134925635856, 114.00...",59.517147,1343.087105,8.689510,5.588881,"{'ONEWEB-0184': {'Earth': (7577.4266183565915,...",557.03100,8PSK,222.498875,QPSK
3,10,<Time tt=2460149.5077451854>,London,"(6365.07943731829, 303.6775402726618, 38.75104...",ONEWEB-0368,"(7554.824618060235, 302.34220135575754, 36.214...","(1232.6360145111812, 111.4034356617881, 156.61...",73.631193,1232.636015,9.943374,7.038232,"{'ONEWEB-0204': {'Earth': (7576.698282476043, ...",659.30025,8PSK,247.579500,8PSK
4,10,<Time tt=2460149.5077451854>,Birmingham,"(6364.724188592993, 301.9223819894903, 37.7670...",ONEWEB-0368,"(7554.824618060235, 302.34220135575754, 36.214...","(1205.231476945039, 125.23378812121153, 151.98...",80.275045,1205.231477,10.097670,7.109956,"{'ONEWEB-0204': {'Earth': (7576.698282476043, ...",659.30025,8PSK,247.579500,8PSK


In [ ]:
df.to_pickle("Results.pkl") #Saves
df = pd.read_pickle("Results.pkl") #Loads